In [1]:
### NQF_WilsonDairy_HydrologicalMonitoring_v1.py
### Version: 6/8/2021
### Author: Khem So, khem_so@fws.gov, (503) 231-6839
### Abstract: This Python 3 script pulls data from the Wilson Dairy Hydrological Data V2 ArcGIS Online feature service (collected via Survey123) and summarizes the data to match an Excel template.

In [2]:
import arcpy
import pandas as pd
from arcgis import GIS
import time, os, fnmatch, shutil
import openpyxl

In [3]:
### ArcGIS Online stores date-time information in UTC by default. This function uses the pytz package to convert time zones and can be used to convert from UTC ("UTC") to localized time. For example, localized "US/Pacific" is either Pacific Standard Time UTC-8 or Pacific Daylight Time UTC-7 depending upon time of year.
from datetime import datetime
from pytz import timezone
def change_timezone_of_field(df, source_date_time_field, new_date_time_field_suffix, source_timezone, new_timezone):
    """Returns the values in *source_date_time_field* with its timezone converted to a new timezone within a new field *new_date_time_field*
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field_suffix: Suffix appended to the end of the name of the source datetime field. This is used to create the new date time field name.
    : param source_timezone: The name of the source timezone
    : param new_timezone: The name of the converted timezone. For possible values, see https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568
    """
    # Define the source timezone in the source_date_time_field
    df[source_date_time_field] = df[source_date_time_field].dt.tz_localize(source_timezone)
    # Define the name of the new date time field
    new_date_time_field = source_date_time_field + new_date_time_field_suffix
    # Convert the datetime in the source_date_time_field to the new timezone in a new field called new_date_time_field
    df[new_date_time_field] = df[source_date_time_field].dt.tz_convert(new_timezone)

In [4]:
### Alternative to change_timezone_of_field function. This function assumes that a source datetime field is UTC and then will calculate a UTC offset (number of hours) on a new datetime field.
from datetime import datetime, timedelta, timezone
def utc_offset_timezone_of_field(df, source_date_time_field, new_date_time_field_suffix, new_timezone_utc_offset):
    """Returns the values in *source_date_time_field* with its timezone converted to a new timezone within a new field *new_date_time_field*
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field_suffix: Suffix appended to the end of the name of the source datetime field. This is used to create the new date time field name.
    : param new_timezone_utc_offset: Number of hours offset from UTC. For example Pacific Standard Time is -8.
    """
    # Define the UTC offset
    offset_tz = timezone(timedelta(hours=new_timezone_utc_offset))
    offset_td = timedelta(hours=new_timezone_utc_offset)
    # Define the name of the new date time field
    new_date_time_field = source_date_time_field + new_date_time_field_suffix
    # Create a temporary offset field
    df['Offset_temp'] = offset_td
    # Calculate the new_date_time_field based arithmetic on the source_date_time_field
    df[new_date_time_field] = df[source_date_time_field] + df['Offset_temp']
    # Define timezone for new_date_time_field
    df[new_date_time_field] = df[new_date_time_field].dt.tz_localize(offset_tz)
    # Define the source timezone in the source_date_time_field
    df[source_date_time_field] = df[source_date_time_field].dt.tz_localize('UTC')

In [5]:
### Allow authentication via login to U.S. Fish & Wildlife Service ArcGIS Online account via ArcGIS Pro
gis = GIS("pro")

In [6]:
### Enter path for local file saving
# uncomment next line to use ArcGIS interface, otherwise hard coding out_workspace
# out_workspace = arcpy.GetParameterAsText(0)
out_workspace = "C:/Users/kso/Desktop/"

In [7]:
### Create timestamp for file naming
t = time.localtime()
timestamp = time.strftime('%Y-%m-%d_%H%M', t)

In [8]:
### Paths to ArcGIS Online data
# To populate Service ItemId, go to Feature Service webpage and in bottom right corner, click on the View link.
# Current Feature Service webpage: https://fws.maps.arcgis.com/home/item.html?id=c6b4f8f33b804dea8c2232c3d7786e9f
ServiceItemID = gis.content.get("c6b4f8f33b804dea8c2232c3d7786e9f")

In [9]:
### There are separate methods for pulling spatial versus non-spatial data into Python. Spatial layers will become Spatially Enabled DataFrame objects. 
## Define variables pointing to spatial layers
WilsonHydroMonitoringLyr = ServiceItemID.layers[0]
## Create Spatially Enabled DataFrame objects
sedfHydroAllData = pd.DataFrame.spatial.from_layer(WilsonHydroMonitoringLyr)

In [10]:
list(sedfHydroAllData.columns)

['objectid',
 'globalid',
 'CreationDate',
 'Creator',
 'EditDate',
 'Editor',
 'observers',
 'observers_other',
 'todays_date',
 'Measurement_Type',
 'what_are_you_measuring_other',
 'creek_gauge_ft',
 'UTC_Time_Measured_4',
 'Location__logger',
 'UTC_Time_Measured_1',
 'depth_to_water_level_inside_wel',
 'depth_to_water_outside_well_ft',
 'depth_to_ground_level_inside_we',
 'depth_to_ground_outside_well_ft',
 'UTC_Time_Measured_2',
 'staff_gauge_ht_ft',
 'data_downloaded',
 'logger_serial',
 'time_logger_pulled_UTC_time',
 'time_logger_re_deployed_UTC_t',
 'battery',
 'Location_culvert',
 'Depth_to_H20_fr0m_top_culvrt_ft',
 'UTC_Time_Measured_3',
 'additional_notes',
 'Measurement_Type_other',
 'Location__logger_other',
 'SHAPE']

In [11]:
### Create Metadata sheet
# Initialize blank metadata dataframe and add columns
metadata = pd.DataFrame(columns = ['Sheet', 'Description'])

# Add records to metadata dataframe using the .loc function
metadata.loc[0] = ["Staff Gage","Nisqually NWR Dempsey Creek Staff Gage at Wilson Dairy"]
metadata.loc[1] = ["Piez P1 East","Nisqually NWR Wilson Dairy - Piezometer P1 (East). Tapedown measurements are made from the measuring point marked on the top of the piezometer. Piezometer 1 (East) Stick-up = 2.57"]
metadata.loc[2] = ["Piez P2 Central","Nisqually NWR Wilson Dairy - Piezometer P2 (Central). Tapedown measurements are made from the measuring point marked on the top of the piezometer. Piezometer 2 (Central) Stick-up = 2.81"]
metadata.loc[3] = ["Piez P3 West","Nisqually NWR Wilson Dairy - Piezometer P3 (West). Tapedown measurements are made from the measuring point marked on the top of the piezometer. Piezometer 3 (West) Stick-up = 2.87"]
metadata.loc[4] = ["Culverts","Nisqually NWR Dephi Road Culverts at Wilson Dairy. Measurements are taken from a measuring point marked on the top of each culvert pipe. Culvert 1 (East) Diameter = 4.97. Culvert 2 (Central) Diameter = 6.0. Culvert 3 (West) Diameter = 6.0. "]
metadata.loc[5] = ["Locations & MP Elevations","Nisqually NWR Staff Gage, Piezometer, and Culvert Locations and Measuring Point Elevations at Wilson Dairy. Location Datum: NAD83(2011); Vertical Datum: NAVD88"]
metadata.loc[6] = ["Water Elevation Data, NAVD88","Nisqually NWR Wilson Dairy - Water Elevation Data (NAVD88)"]
metadata.loc[7] = ["WL Data, Gage Datum","Nisqually NWR Wilson Dairy - Water Elevation Data (Gage Datum). 0 on Staff Gage = 138.899"]
metadata.loc[8] = [timestamp + " AGOL Data","ArcGIS Online Data Download"]

In [12]:
### Create Locations and MP Elevations sheet
# Initialize blank dataframe and add columns
locations = pd.DataFrame(columns = ['Site Name', 'Latitude','Longitude', 'Elevation of M.P. (Ft)', 'Notes', 'identifier'])

# Add records to metadata dataframe using the .loc function
locations.loc[0] = ["Dempsey Creek Stilling Well",46.96455946,-123.02094791,141.413,"Stilling well is immediately adjacent to the staff gage (within 2 ft of gage).", ""]
locations.loc[1] = ["Dempsey Creek Staff Gage",46.96455946,-123.02094791,138.899,"GPS coordinates from stilling well are utilized here due to the proximity of well and gage. Elevation value is for 0 on the staff gage and is based upon the average of two RTK shots", "dempsey_creek_gauge"]
locations.loc[2] = ["Piezometer 1 (East)",46.96451927,-123.02189070,142.905,"", "piezometer_1_east"]
locations.loc[3] = ["Piezometer 2 (Central)",46.96443652,-123.02280777,143.152,"", "piezometer_2_central"]
locations.loc[4] = ["Piezometer 3 (West)",46.96405927,-123.02394417,144.686,"", "piezometer_3_west"]
locations.loc[5] = ["Culvert 1 (East)",46.96475993,-123.02207761,141.484,"", "culvert_1_east"]
locations.loc[6] = ["Culvert 2 (Central)",46.96475841,-123.02219930,142.315,"This culvert is the western-most of two culverts at this location.", "culvert_2_middle"]
locations.loc[7] = ["Culvert 3 (West)",46.96476425,-123.02310604,143.114,"This culvert is the western-most of two culverts at this location.", "culvert_3_west"]
locations

,Site Name,Latitude,Longitude,Elevation of M.P. (Ft),Notes,identifier
0,Dempsey Creek Stilling Well,46.964559,-123.020948,141.413,Stilling well is immediately adjacent to the s...,
1,Dempsey Creek Staff Gage,46.964559,-123.020948,138.899,GPS coordinates from stilling well are utilize...,dempsey_creek_gauge
2,Piezometer 1 (East),46.964519,-123.021891,142.905,,piezometer_1_east
3,Piezometer 2 (Central),46.964437,-123.022808,143.152,,piezometer_2_central
4,Piezometer 3 (West),46.964059,-123.023944,144.686,,piezometer_3_west
5,Culvert 1 (East),46.964760,-123.022078,141.484,,culvert_1_east
6,Culvert 2 (Central),46.964758,-123.022199,142.315,This culvert is the western-most of two culver...,culvert_2_middle
7,Culvert 3 (West),46.964764,-123.023106,143.114,This culvert is the western-most of two culver...,culvert_3_west


In [13]:
# ## Alternative 1: Use change_timezone_of_field function to convert all datetime fields in dataframe from UTC to Pacific within new field with _Pacific suffix
# for col in sedfHydroAllData.columns:
#     if sedfHydroAllData[col].dtype == 'datetime64[ns]':
#         change_timezone_of_field(sedfHydroAllData, col, "_Pacific", "UTC", "US/Pacific")

In [14]:
## Alternative 2: Use utc_offset_timezone_of_field function to convert all datetime fields in dataframe from UTC to PST within new field with _PST suffix
for col in sedfHydroAllData.columns:
    if sedfHydroAllData[col].dtype == 'datetime64[ns]':
        utc_offset_timezone_of_field(sedfHydroAllData, col, '_PST', -8)

In [15]:
# Check data types
sedfHydroAllData.dtypes

objectid                                                 int64
globalid                                                object
CreationDate                               datetime64[ns, UTC]
Creator                                                 object
EditDate                                   datetime64[ns, UTC]
Editor                                                  object
observers                                               object
observers_other                                         object
todays_date                                datetime64[ns, UTC]
Measurement_Type                                        object
what_are_you_measuring_other                            object
creek_gauge_ft                                         float64
UTC_Time_Measured_4                        datetime64[ns, UTC]
Location__logger                                        object
UTC_Time_Measured_1                        datetime64[ns, UTC]
depth_to_water_level_inside_wel                        

In [16]:
# Check datetime data
sedfHydroAllData.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]'])

,CreationDate,EditDate,todays_date,UTC_Time_Measured_4,UTC_Time_Measured_1,UTC_Time_Measured_2,time_logger_pulled_UTC_time,time_logger_re_deployed_UTC_t,UTC_Time_Measured_3,CreationDate_PST,EditDate_PST,todays_date_PST,UTC_Time_Measured_4_PST,UTC_Time_Measured_1_PST,UTC_Time_Measured_2_PST,time_logger_pulled_UTC_time_PST,time_logger_re_deployed_UTC_t_PST,UTC_Time_Measured_3_PST
0,2020-02-19 01:27:35.359999895+00:00,2020-03-04 20:01:31.436000109+00:00,2020-02-18 20:00:00+00:00,2020-02-18 20:30:00+00:00,NaT,NaT,NaT,NaT,NaT,2020-02-18 17:27:35.359999895-08:00,2020-03-04 12:01:31.436000109-08:00,2020-02-18 12:00:00-08:00,2020-02-18 12:30:00-08:00,NaT,NaT,NaT,NaT,NaT
1,2020-02-19 01:27:41.535000086+00:00,2020-02-19 01:27:41.535000086+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,2020-02-18 20:34:09.976000071+00:00,2020-02-18 20:33:00+00:00,2020-02-18 20:38:00+00:00,NaT,2020-02-18 17:27:41.535000086-08:00,2020-02-18 17:27:41.535000086-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,2020-02-18 12:34:09.976000071-08:00,2020-02-18 12:33:00-08:00,2020-02-18 12:38:00-08:00,NaT
2,2020-02-19 01:27:44.867000103+00:00,2020-02-19 01:27:44.867000103+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,NaT,2020-02-18 20:26:40.049999952+00:00,2020-02-18 20:27:00+00:00,NaT,2020-02-18 17:27:44.867000103-08:00,2020-02-18 17:27:44.867000103-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,NaT,2020-02-18 12:26:40.049999952-08:00,2020-02-18 12:27:00-08:00,NaT
3,2020-02-19 01:27:48.403000116+00:00,2020-02-19 01:27:48.403000116+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,NaT,2020-02-18 20:23:44.302999973+00:00,2020-02-18 20:24:00+00:00,NaT,2020-02-18 17:27:48.403000116-08:00,2020-02-18 17:27:48.403000116-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,NaT,2020-02-18 12:23:44.302999973-08:00,2020-02-18 12:24:00-08:00,NaT
4,2020-02-19 01:27:55.072999954+00:00,2020-02-19 05:24:42.161000013+00:00,2020-02-18 20:00:00+00:00,NaT,2020-02-18 20:00:34.260999918+00:00,NaT,2020-02-18 20:03:00+00:00,2020-02-18 20:11:21.598999977+00:00,NaT,2020-02-18 17:27:55.072999954-08:00,2020-02-18 21:24:42.161000013-08:00,2020-02-18 12:00:00-08:00,NaT,2020-02-18 12:00:34.260999918-08:00,NaT,2020-02-18 12:03:00-08:00,2020-02-18 12:11:21.598999977-08:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2021-06-03 13:24:29.697000027+00:00,2021-06-03 13:24:29.697000027+00:00,2021-05-17 23:58:09.528000115+00:00,NaT,NaT,NaT,NaT,NaT,2021-05-17 23:58:20.746999979+00:00,2021-06-03 05:24:29.697000027-08:00,2021-06-03 05:24:29.697000027-08:00,2021-05-17 15:58:09.528000115-08:00,NaT,NaT,NaT,NaT,NaT,2021-05-17 15:58:20.746999979-08:00
211,2021-06-03 13:24:46.849999905+00:00,2021-06-03 13:24:46.849999905+00:00,2021-05-26 23:34:45.665999889+00:00,2021-05-26 23:35:00+00:00,NaT,NaT,NaT,NaT,NaT,2021-06-03 05:24:46.849999905-08:00,2021-06-03 05:24:46.849999905-08:00,2021-05-26 15:34:45.665999889-08:00,2021-05-26 15:35:00-08:00,NaT,NaT,NaT,NaT,NaT
212,2021-06-03 13:24:56.306999922+00:00,2021-06-03 13:24:56.306999922+00:00,2021-05-26 23:39:50.819999933+00:00,NaT,NaT,NaT,NaT,NaT,2021-05-26 23:40:00+00:00,2021-06-03 05:24:56.306999922-08:00,2021-06-03 05:24:56.306999922-08:00,2021-05-26 15:39:50.819999933-08:00,NaT,NaT,NaT,NaT,NaT,2021-05-26 15:40:00-08:00
213,2021-06-03 13:25:09.088999987+00:00,2021-06-03 13:25:09.088999987+00:00,2021-05-26 19:00:00+00:00,NaT,NaT,NaT,NaT,NaT,2021-05-26 23:43:00+00:00,2021-06-03 05:25:09.088999987-08:00,2021-06-03 05:25:09.088999987-08:00,2021-05-26 11:00:00-08:00,NaT,NaT,NaT,NaT,NaT,2021-05-26 15:43:00-08:00


In [17]:
### Archive AGOL Data
sedfHydroAllDataArchive = sedfHydroAllData.copy()
# Convert Python date time into format Excel can read more easily
archive_dt_field_list = sedfHydroAllDataArchive.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]'])
for col in archive_dt_field_list:
    sedfHydroAllDataArchive[col] = sedfHydroAllDataArchive[col].dt.strftime('%m/%d/%Y %H:%M:%S %Z%z')

In [18]:
# Define sort order
sedfHydroAllData = sedfHydroAllData.sort_values(by=['todays_date_PST'])

In [19]:
# Convert Python date time into format Excel can read more easily
datefield_list = sedfHydroAllData[['todays_date_PST']]

for col in datefield_list:
    sedfHydroAllData[col] = sedfHydroAllData[col].dt.strftime('%m/%d/%Y')

timefield_list = sedfHydroAllData[['UTC_Time_Measured_4', 'UTC_Time_Measured_1', 'UTC_Time_Measured_2', 'time_logger_pulled_UTC_time', 'time_logger_re_deployed_UTC_t', 'UTC_Time_Measured_3', 'UTC_Time_Measured_4_PST', 'UTC_Time_Measured_1_PST', 'UTC_Time_Measured_2_PST', 'time_logger_pulled_UTC_time_PST', 'time_logger_re_deployed_UTC_t_PST', 'UTC_Time_Measured_3_PST']]

for col in timefield_list:
    sedfHydroAllData[col] = sedfHydroAllData[col].dt.strftime('%H:%M')

In [20]:
### Subset data for Dempsey Creek Staff Gage at Wilson Dairy, Nisqually NWR

# Select data based on 2 conditions: measurement type = piezometer and location = dempsey creek gauge OR measurement type = dempsey creek level quick survey
# Select columns and reset in desired order
sedfHydroStaffGageData = sedfHydroAllData.loc[((sedfHydroAllData['Measurement_Type'] == "piezometers") & (sedfHydroAllData['Location__logger'] == "dempsey_creek_gauge")) | ((sedfHydroAllData['Measurement_Type'] == "dempsey_creek_level_quick_surve")), ['todays_date_PST', 'UTC_Time_Measured_4_PST', 'observers', 'observers_other', 'creek_gauge_ft', 'UTC_Time_Measured_2_PST', 'staff_gauge_ht_ft', 'data_downloaded', 'logger_serial', 'time_logger_pulled_UTC_time_PST', 'time_logger_re_deployed_UTC_t_PST', 'battery', 'additional_notes', 'globalid', 'SHAPE']]

# Rename columns
sedfHydroStaffGageData.rename(columns = {'todays_date_PST':'Date', 'UTC_Time_Measured_4_PST':'Time of Measurement 1 (PST)', 'observers':'Observers', 'observers_other':'Other Observers', 'creek_gauge_ft':'Water Level, Arrival (ft)', 'UTC_Time_Measured_2_PST':'Time of Measurement 2 (PST)', 'staff_gauge_ht_ft':'Water Level, Departure (ft)', 'data_downloaded':'Downloaded Data', 'logger_serial':'Logger Serial#', 'time_logger_pulled_UTC_time_PST':'Time Logger Pulled (PST)', 'time_logger_re_deployed_UTC_t_PST':'Time Logger Re-deployed (PST)', 'battery':'Battery', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [21]:
### Subset data for piezometer Piez P1 East at Wilson Dairy, Nisqually NWR

## Select data based on 2 conditions: measurement type = piezometer and location = piezometer_1_east
# Create filter
PiezP1_filter = (sedfHydroAllData['Measurement_Type'] == "piezometers") & (sedfHydroAllData['Location__logger'] == "piezometer_1_east")
# Copy HydroAllData to new dataframe
sedfHydroPiezP1EastData = sedfHydroAllData.copy()
# Create new column for Height of water above HOBO
sedfHydroPiezP1EastData['Height of water above HOBO (Ft; length of stick = 6.20)'] = ''
# Calculate Height of water above HOBO (length of stick = 6.20')
lengthofstickP1 = 6.2

sedfHydroPiezP1EastData.loc[PiezP1_filter, ['Height of water above HOBO (Ft; length of stick = 6.20)']] = lengthofstickP1 - sedfHydroPiezP1EastData['depth_to_water_level_inside_wel']

# Select columns and reset in desired order
sedfHydroPiezP1EastData = sedfHydroPiezP1EastData.loc[PiezP1_filter, ['todays_date_PST', 'UTC_Time_Measured_1_PST', 'observers', 'observers_other', 'depth_to_water_level_inside_wel', 'Height of water above HOBO (Ft; length of stick = 6.20)', 'depth_to_water_outside_well_ft', 'depth_to_ground_level_inside_we', 'depth_to_ground_outside_well_ft', 'data_downloaded', 'logger_serial', 'time_logger_pulled_UTC_time_PST', 'time_logger_re_deployed_UTC_t_PST', 'battery', 'additional_notes', 'globalid', 'SHAPE']]

# Rename columns
sedfHydroPiezP1EastData.rename(columns = {'todays_date_PST':'Date', 'UTC_Time_Measured_1_PST':'Time of Measurement (PST)', 'observers':'Observers', 'observers_other':'Other Observers', 'depth_to_water_level_inside_wel':'Depth to Water Level Inside Well (Ft)', 'depth_to_water_outside_well_ft':'Depth to Water Outside Well (Ft)', 'depth_to_ground_level_inside_we':'Depth to Ground Level Inside Well (Ft)', 'depth_to_ground_outside_well_ft':'Depth to Ground Outside Well (Ft)', 'data_downloaded':'Downloaded Data', 'logger_serial':'Logger Serial#', 'time_logger_pulled_UTC_time_PST':'Time Logger Pulled (PST)', 'time_logger_re_deployed_UTC_t_PST':'Time Logger Re-deployed (PST)', 'battery':'Battery', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [22]:
### Subset data for piezometer Piez P2 Central at Wilson Dairy, Nisqually NWR

## Select data based on 2 conditions: measurement type = piezometer and location = piezometer_2_central
# Create filter
PiezP2_filter = (sedfHydroAllData['Measurement_Type'] == "piezometers") & (sedfHydroAllData['Location__logger'] == "piezometer_2_central")
# Copy HydroAllData to new dataframe
sedfHydroPiezP2CentralData = sedfHydroAllData.copy()
# Create new column for Height of water above HOBO
sedfHydroPiezP2CentralData['Height of water above HOBO (Ft; length of stick = 6.20)'] = ''
# Calculate Height of water above HOBO (length of stick = 6.20')
lengthofstickP2 = 6.2

sedfHydroPiezP2CentralData.loc[PiezP2_filter, ['Height of water above HOBO (Ft; length of stick = 6.20)']] = lengthofstickP2 - sedfHydroPiezP2CentralData['depth_to_water_level_inside_wel']

# Select columns and reset in desired order
sedfHydroPiezP2CentralData = sedfHydroPiezP2CentralData.loc[PiezP2_filter, ['todays_date_PST', 'UTC_Time_Measured_1_PST', 'observers', 'observers_other', 'depth_to_water_level_inside_wel', 'Height of water above HOBO (Ft; length of stick = 6.20)', 'depth_to_water_outside_well_ft', 'depth_to_ground_level_inside_we', 'depth_to_ground_outside_well_ft', 'data_downloaded', 'logger_serial', 'time_logger_pulled_UTC_time_PST', 'time_logger_re_deployed_UTC_t_PST', 'battery', 'additional_notes', 'globalid', 'SHAPE']]

# Rename columns
sedfHydroPiezP2CentralData.rename(columns = {'todays_date_PST':'Date', 'UTC_Time_Measured_1_PST':'Time of Measurement (PST)', 'observers':'Observers', 'observers_other':'Other Observers', 'depth_to_water_level_inside_wel':'Depth to Water Level Inside Well (Ft)', 'depth_to_water_outside_well_ft':'Depth to Water Outside Well (Ft)', 'depth_to_ground_level_inside_we':'Depth to Ground Level Inside Well (Ft)', 'depth_to_ground_outside_well_ft':'Depth to Ground Outside Well (Ft)', 'data_downloaded':'Downloaded Data', 'logger_serial':'Logger Serial#', 'time_logger_pulled_UTC_time_PST':'Time Logger Pulled (PST)', 'time_logger_re_deployed_UTC_t_PST':'Time Logger Re-deployed (PST)', 'battery':'Battery', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [23]:
### Subset data for piezometer Piez P3 West at Wilson Dairy, Nisqually NWR

## Select data based on 2 conditions: measurement type = piezometer and location = piezometer_3_west
# Create filter
PiezP3_filter = (sedfHydroAllData['Measurement_Type'] == "piezometers") & (sedfHydroAllData['Location__logger'] == "piezometer_3_west")
# Copy HydroAllData to new dataframe
sedfHydroPiezP3WestData = sedfHydroAllData.copy()
# Create new column for Height of water above HOBO
sedfHydroPiezP3WestData['Height of water above HOBO (Ft; length of stick = 5.65)'] = ''
# Calculate Height of water above HOBO (length of stick = 5.65')
lengthofstickP3 = 5.65

sedfHydroPiezP3WestData.loc[PiezP3_filter, ['Height of water above HOBO (Ft; length of stick = 5.65)']] = lengthofstickP3 - sedfHydroPiezP3WestData['depth_to_water_level_inside_wel']

# Select columns and reset in desired order
sedfHydroPiezP3WestData = sedfHydroPiezP3WestData.loc[PiezP3_filter, ['todays_date_PST', 'UTC_Time_Measured_1_PST', 'observers', 'observers_other', 'depth_to_water_level_inside_wel', 'Height of water above HOBO (Ft; length of stick = 5.65)', 'depth_to_water_outside_well_ft', 'depth_to_ground_level_inside_we', 'depth_to_ground_outside_well_ft', 'data_downloaded', 'logger_serial', 'time_logger_pulled_UTC_time_PST', 'time_logger_re_deployed_UTC_t_PST', 'battery', 'additional_notes', 'globalid', 'SHAPE']]

# Rename columns
sedfHydroPiezP3WestData.rename(columns = {'todays_date_PST':'Date', 'UTC_Time_Measured_1_PST':'Time of Measurement (PST)', 'observers':'Observers', 'observers_other':'Other Observers', 'depth_to_water_level_inside_wel':'Depth to Water Level Inside Well (Ft)', 'depth_to_water_outside_well_ft':'Depth to Water Outside Well (Ft)', 'depth_to_ground_level_inside_we':'Depth to Ground Level Inside Well (Ft)', 'depth_to_ground_outside_well_ft':'Depth to Ground Outside Well (Ft)', 'data_downloaded':'Downloaded Data', 'logger_serial':'Logger Serial#', 'time_logger_pulled_UTC_time_PST':'Time Logger Pulled (PST)', 'time_logger_re_deployed_UTC_t_PST':'Time Logger Re-deployed (PST)', 'battery':'Battery', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [24]:
### Subset data for culverts

## Select data based on 1 condition: measurement type = culverts
sedfHydroCulvertsData = sedfHydroAllData.loc[(sedfHydroAllData['Measurement_Type'] == "culverts"), ['Location_culvert', 'todays_date_PST', 'UTC_Time_Measured_3_PST', 'observers', 'observers_other', 'Depth_to_H20_fr0m_top_culvrt_ft', 'additional_notes', 'globalid', 'SHAPE', 'todays_date']]

# Define sort order
sedfHydroCulvertsData = sedfHydroCulvertsData.sort_values(by=['Location_culvert', 'todays_date'])

# Drop todays_date
del sedfHydroCulvertsData['todays_date']

# Rename columns
sedfHydroCulvertsData.rename(columns = {'Location_culvert':'Culvert', 'todays_date_PST':'Date', 'UTC_Time_Measured_3_PST':'Time of Measurement (PST)', 'observers':'Observers', 'observers_other':'Other Observers', 'Depth_to_H20_fr0m_top_culvrt_ft':'Depth to Water Level from Culvert Top (Ft)', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [25]:
### Calculate water elevation data at staff gage (NAVD88 and Gage Datum)

## Get unique combinations of staff gage reading date and water level
# Copy sedfHydroStaffGageData
uniqueStaffGage = sedfHydroStaffGageData.copy()

# Fill NaN with 0
uniqueStaffGage[['Water Level, Arrival (ft)', 'Water Level, Departure (ft)']] = uniqueStaffGage[['Water Level, Arrival (ft)', 'Water Level, Departure (ft)']].fillna(0)
# Data structured such that Water Level Arrival and Departure mutually exclusive so summing them give the single water level value
uniqueStaffGage['Water Level'] = uniqueStaffGage['Water Level, Arrival (ft)'] + uniqueStaffGage['Water Level, Departure (ft)'] 

# Convert Date to Python date field for sorting
uniqueStaffGage['Date'] = pd.to_datetime(uniqueStaffGage['Date'], format = '%m/%d/%Y')

# Subset to Date and Water Level fields
uniqueStaffGage = uniqueStaffGage[['Date', 'Water Level']]

# Group by to select unique combinations of Date and Water Level
uniqueStaffGage = uniqueStaffGage.groupby(['Date', 'Water Level']).size().reset_index().rename(columns={0:'count'})

### Calculate Water Elevation Data (NAVD88 and Gage Datum)
uniqueStaffGage['Staff (NAVD88)'] = locations.loc[1, 'Elevation of M.P. (Ft)'] + uniqueStaffGage['Water Level']
uniqueStaffGage['Staff (Gage Datum)'] = uniqueStaffGage['Staff (NAVD88)'] - locations.loc[1, 'Elevation of M.P. (Ft)'] 

# Sort by date
uniqueStaffGage = uniqueStaffGage.sort_values(by=['Date'])

In [26]:
### Calculate water elevation data at piezometers (NAVD88 and Gage Datum)
# PiezP1East
uniquePiezP1East = sedfHydroPiezP1EastData.copy()
uniquePiezP1East['Water Inside P1 (NAVD88)'] = locations.loc[2, 'Elevation of M.P. (Ft)'] - uniquePiezP1East['Depth to Water Level Inside Well (Ft)']
uniquePiezP1East['Date'] = pd.to_datetime(uniquePiezP1East['Date'], format = '%m/%d/%Y')
uniquePiezP1East = uniquePiezP1East[['Date', 'Water Inside P1 (NAVD88)']]
uniquePiezP1East.dropna(subset=['Water Inside P1 (NAVD88)'], inplace= True)
uniquePiezP1East = uniquePiezP1East.sort_values(by=['Date'])
uniquePiezP1East['Water Inside P1 (Gage Datum)'] = uniquePiezP1East['Water Inside P1 (NAVD88)'] - locations.loc[1, 'Elevation of M.P. (Ft)'] 
# PiezP2Central
uniquePiezP2Central = sedfHydroPiezP2CentralData.copy()
uniquePiezP2Central['Water Inside P2 (NAVD88)'] = locations.loc[3, 'Elevation of M.P. (Ft)'] - uniquePiezP2Central['Depth to Water Level Inside Well (Ft)']
uniquePiezP2Central['Date'] = pd.to_datetime(uniquePiezP2Central['Date'], format = '%m/%d/%Y')
uniquePiezP2Central = uniquePiezP2Central[['Date', 'Water Inside P2 (NAVD88)']]
uniquePiezP2Central.dropna(subset=['Water Inside P2 (NAVD88)'], inplace= True)
uniquePiezP2Central = uniquePiezP2Central.sort_values(by=['Date'])
uniquePiezP2Central['Water Inside P2 (Gage Datum)'] = uniquePiezP2Central['Water Inside P2 (NAVD88)'] - locations.loc[1, 'Elevation of M.P. (Ft)'] 
# PiezP3West
uniquePiezP3West = sedfHydroPiezP3WestData.copy()
uniquePiezP3West['Water Inside P3 (NAVD88)'] = locations.loc[4, 'Elevation of M.P. (Ft)'] - uniquePiezP3West['Depth to Water Level Inside Well (Ft)']
uniquePiezP3West['Date'] = pd.to_datetime(uniquePiezP3West['Date'], format = '%m/%d/%Y')
uniquePiezP3West = uniquePiezP3West[['Date', 'Water Inside P3 (NAVD88)']]
uniquePiezP3West.dropna(subset=['Water Inside P3 (NAVD88)'], inplace= True)
uniquePiezP3West = uniquePiezP3West.sort_values(by=['Date'])
uniquePiezP3West['Water Inside P3 (Gage Datum)'] = uniquePiezP3West['Water Inside P3 (NAVD88)'] - locations.loc[1, 'Elevation of M.P. (Ft)'] 

In [27]:
### Combine data
combine = pd.merge(uniqueStaffGage, uniquePiezP1East, how="outer", left_on="Date", right_on="Date")
del combine['count']
combine = pd.merge(combine, uniquePiezP2Central, how="outer", left_on="Date", right_on="Date")
combine = pd.merge(combine, uniquePiezP3West, how="outer", left_on="Date", right_on="Date")
combine


,Date,Water Level,Staff (NAVD88),Staff (Gage Datum),Water Inside P1 (NAVD88),Water Inside P1 (Gage Datum),Water Inside P2 (NAVD88),Water Inside P2 (Gage Datum),Water Inside P3 (NAVD88),Water Inside P3 (Gage Datum)
0,2020-02-18,1.69,140.589,1.69,140.615,1.716,142.202,3.303,142.246,3.347
1,2020-02-21,1.40,140.299,1.40,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-27,1.29,140.189,1.29,140.415,1.516,141.552,2.653,142.026,3.127
3,2020-03-06,1.35,140.249,1.35,140.305,1.406,141.382,2.483,142.966,4.067
4,2020-03-21,0.78,139.679,0.78,140.105,1.206,141.162,2.263,141.776,2.877
5,2020-05-22,0.56,139.459,0.56,139.445,0.546,140.542,1.643,141.396,2.497
6,2020-05-27,0.45,139.349,0.45,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-06-03,0.30,139.199,0.30,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-06-10,0.99,139.889,0.99,140.105,1.206,140.322,1.423,141.246,2.347
9,2020-06-19,0.55,139.449,0.55,139.415,0.516,140.272,1.373,141.116,2.217


In [28]:
### Create export paths
writer = pd.ExcelWriter(os.path.join(out_workspace,(timestamp + '_WilsonDairyHydro.xlsx')))
metadata.to_excel(writer, 'Metadata')
sedfHydroStaffGageData.to_excel(writer, 'Staff Gage')
sedfHydroPiezP1EastData.to_excel(writer, 'Piez P1 East')
sedfHydroPiezP2CentralData.to_excel(writer, 'Piez P2 Central')
sedfHydroPiezP3WestData.to_excel(writer, 'Piez P3 West')
sedfHydroCulvertsData.to_excel(writer, 'Culverts')
locations.to_excel(writer, 'Locations & MP Elevations')
combine.to_excel(writer, 'Water Elevation Data')
sedfHydroAllDataArchive.to_excel(writer, (timestamp + " AGOL Data"))
writer.save()